## Use OpenCV to get frames from url

In [22]:
import cv2

# HLS playlist URL
hls_url = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"

cap = cv2.VideoCapture(hls_url)

frame_count = 0
frame_skip = 100  # Skip every 5 frames

if not cap.isOpened():
    print("Error: Could not open the HLS stream.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Frame not received, ending stream")
            break
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue
        # Process your frame (e.g., people counting) here

        # For debugging, display the frame
        cv2.imshow("HLS Stream", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


## Count People in Web Cam Using YOLOv8

In [25]:
import cv2
from ultralytics import YOLO
import numpy as np
import torch

# Import the necessary classes - this is a new issue with pytorch 2.6, could use 2.4 and not have to import all of these layers and add them to the globals list
from ultralytics.nn.tasks import DetectionModel  # Already imported for YOLOv8 models
from torch.nn.modules.container import Sequential    # For Sequential layers
from ultralytics.nn.modules.conv import Conv         # For Conv layers defined by Ultralytics
from torch.nn.modules.conv import Conv2d              # For PyTorch's Conv2d layer
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.nn.modules.activation import SiLU               # PyTorch's SiLU activation
from ultralytics.nn.modules.block import C2f                       # Ultralytics' C2f block
from torch.nn.modules.container import ModuleList
from ultralytics.nn.modules.block import Bottleneck
from ultralytics.nn.modules.block import SPPF
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.upsampling import Upsample
from ultralytics.nn.modules.conv import Concat
from ultralytics.nn.modules.head import Detect
from ultralytics.nn.modules.block import DFL
torch.serialization.add_safe_globals([
    DetectionModel, Sequential, Conv, Conv2d, BatchNorm2d, SiLU, C2f, ModuleList,
    Bottleneck, SPPF, MaxPool2d, Upsample, Concat, Detect, DFL
])



device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the pretrained YOLOv8 model and move it to the appropriate device
model = YOLO("yolov8n.pt").to(device)

# Define the HLS stream URL (the direct stream URL you extracted)
hls_url = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"

# Open the video stream
cap = cv2.VideoCapture(hls_url)
if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

frame_skip = 20  # Process every 20th frame
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    frame_count += 1

    # Skip frames until we hit the desired interval
    if frame_count % frame_skip != 0:
        continue

    # Run YOLOv8 inference on the current frame.
    results = model(frame)
    
    # We'll use a copy of the frame to draw annotations.
    annotated_frame = frame.copy()
    people_count = 0

    # Process each detection result
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.data.cpu().numpy()  # shape: (num_boxes, 6)
            for box in boxes:
                x1, y1, x2, y2, conf, cls = box
                # In COCO, the 'person' class typically has an id of 0.
                if int(cls) >= 0:
                    bbox_color = (0, 255, 0)  # Green
                    if cls != 0:
                        bbox_color = (255, 0, 0)
                    cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), bbox_color, 2)
                    people_count += 1
                    # Draw the bounding box and label on the frame.
                    cv2.putText(
                        annotated_frame,
                        f"{int(cls)} {conf:.2f}",
                        (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2,
                    )
    
    # Overlay the people count on the frame.
    cv2.putText(
        annotated_frame,
        f"People Count: {people_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
    )
    
    # Display the annotated frame
    cv2.imshow("YOLOv8 People Counting", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the stream and close windows
cap.release()
cv2.destroyAllWindows()


Using device: cuda

0: 384x640 9 persons, 21.5ms
Speed: 4.1ms preprocess, 21.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 20.6ms
Speed: 11.6ms preprocess, 20.6ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 25.2ms
Speed: 5.7ms preprocess, 25.2ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 23.7ms
Speed: 9.9ms preprocess, 23.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 50.1ms
Speed: 5.0ms preprocess, 50.1ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 57.4ms
Speed: 15.4ms preprocess, 57.4ms inference, 10.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 21.5ms
Speed: 5.3ms preprocess, 21.5ms inference, 5.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 19.0ms
Speed: 6.1ms preprocess, 19.0ms inference, 2.1ms postproces

## Create a dataset we can use to fine tune the model for counting people on this web cam
Saves every 1000th frame into the dataset/images/train folder and the associated YOLOv8 detected people annotations into the dataset/labels/train folder

Idea is we then go through these images after we gather a lot and improve upon the annotations. Then we fine tune the YOLO model with this data.

In [31]:
import os
import cv2
import time
from ultralytics import YOLO
import torch

# Set up safe globals for PyTorch 2.6+ (include only if necessary)
from ultralytics.nn.tasks import DetectionModel
from torch.nn.modules.container import Sequential, ModuleList
from ultralytics.nn.modules.conv import Conv, Concat
from torch.nn.modules.conv import Conv2d
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.nn.modules.activation import SiLU
from ultralytics.nn.modules.block import C2f, Bottleneck, SPPF, DFL
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.upsampling import Upsample
from ultralytics.nn.modules.head import Detect
torch.serialization.add_safe_globals([
    DetectionModel, Sequential, Conv, Conv2d, BatchNorm2d, SiLU, C2f, ModuleList,
    Bottleneck, SPPF, MaxPool2d, Upsample, Concat, Detect, DFL
])

# Adjustable parameters
WEBCAM_URL = "https://streamer4.brownrice.com/camdensnowbowl1/camdensnowbowl1.stream/main_playlist.m3u8"
FRAME_INTERVAL = 1000  # Process every 1000th frame
CONF_THRESHOLD = 0.4   # Confidence threshold
IMG_DIR = 'dataset/images/train'
LABEL_DIR = 'dataset/labels/train'


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load YOLOv8 model
model = YOLO("yolov8n.pt").to(device)

cap = cv2.VideoCapture(WEBCAM_URL)
if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

frame_count = 0

while True:
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame_count += 1
    # Process only every FRAME_INTERVAL frame
    if frame_count % FRAME_INTERVAL != 1:
        continue

    # Save the raw frame
    timestamp = int(time.time())
    img_filename = os.path.join(IMG_DIR, f"frame_{frame_count}_{timestamp}.jpg")
    cv2.imwrite(img_filename, frame)
    print(f"Saved image: {img_filename}")

    # Run YOLOv8 inference on the frame
    results = model(frame)
    height, width = frame.shape[:2]

    # Open a .txt file for writing the annotations in YOLO format
    txt_filename = os.path.splitext(img_filename)[0] + ".txt"
    txt_filename = txt_filename.replace("images", "labels")
    with open(txt_filename, "w") as f:
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes.data.cpu().numpy()  # each row: [x1, y1, x2, y2, conf, cls]
                for box in boxes:
                    x1, y1, x2, y2, conf, cls = box
                    if conf >= CONF_THRESHOLD and int(cls) == 0:  # Only person (class 0)
                        # Convert bounding box to YOLO format (normalized)
                        x_center = ((x1 + x2) / 2.0) / width
                        y_center = ((y1 + y2) / 2.0) / height
                        bbox_width = (x2 - x1) / width
                        bbox_height = (y2 - y1) / height
                        # Write annotation line: class x_center y_center width height
                        f.write(f"0 {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")
    print(f"Saved annotations: {txt_filename}")

    # Optionally, display the frame (with no annotations drawn)
    cv2.imshow("Dataset Collection", frame)


cap.release()
cv2.destroyAllWindows()


Using device: cuda
Saved image: dataset/images/train/frame_1_1738874235.jpg

0: 384x640 4 persons, 64.1ms
Speed: 13.7ms preprocess, 64.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Saved annotations: dataset/labels/train/frame_1_1738874235.txt
Saved image: dataset/images/train/frame_1001_1738874278.jpg

0: 384x640 8 persons, 58.6ms
Speed: 6.7ms preprocess, 58.6ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
Saved annotations: dataset/labels/train/frame_1001_1738874278.txt
